## Quantitaive and qualitative data inspection

In [1]:
# libraries 
import numpy as np
import h5py
import ipywidgets as widgets
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.animation as animation
from IPython.display import display
import os
import sys
sys.path.append('../../src')

from utils.evaluate_utils import get_indices
# from prepare_data.kspace_sampling import adjust_image_size_centered

### Qualitative analysis

In [ ]:
data_dir = '../../data/CARDIAC'

dataset1 = 'M2_2mm_step2_cs_invivoP04_hr_corrected.h5'
dataset2 = 'M2_2mm_step2_cs_invivoP04_hr.h5'

with h5py.File(os.path.join(data_dir, dataset1), 'r') as f:
    u1 = f['u'][:]
    v1 = f['v'][:]
    w1 = f['w'][:]
    mag1 = f['mag_u'][:]
with h5py.File(os.path.join(data_dir, dataset2), 'r') as f:
    u2 = f['u'][:]
    v2 = f['v'][:]
    w2 = f['w'][:]
    mag_2 = f['mag_u'][:]


crop_left = int(np.ceil((u2.shape[1] - u1.shape[1])/2))
crop_right = int(np.floor((u2.shape[1] - u1.shape[1])/2))

u2 = u2[:, crop_left:-crop_right, :, :]

print('u1 shape:', u1.shape)
print('v1 shape:', v1.shape)
print('w1 shape:', w1.shape)
print('u2 shape:', u2.shape)
print('v2 shape:', v2.shape)
print('w2 shape:', w2.shape)


T, X, Y, Z = u1.shape
# print difference

print('u1 - u2:', np.sum(np.abs(u1 - u2)))
# print('v1 - v2:', np.sum(np.abs(v1 - v2)))
# print('w1 - w2:', np.sum(np.abs(w1 - w2)))

@widgets.interact(frame=(0, T-1), x=(0, X-1), axis=[0, 1, 2])
def visualize_lr(frame=0, x=0, axis=0):
    plt.figure(figsize=(12, 7))
    N = 4
    idxs_hr = get_indices(frame, axis, x)

    plt.subplot(1, N, 1)
    plt.imshow(u1[idxs_hr])
    plt.title('u1')
    plt.axis('off')

    plt.subplot(1, N, 2)
    plt.imshow(u2[idxs_hr])
    plt.title('u2')
    plt.axis('off')

    plt.subplot(1, N, 3)
    plt.imshow(mag1[idxs_hr])
    plt.title('mag 1')
    plt.axis('off')

    plt.subplot(1, N, 4)
    plt.imshow(mag_2[idxs_hr])
    plt.title('mag 2')
    plt.axis('off')




u1 shape: (50, 72, 60, 96)
v1 shape: (50, 72, 60, 96)
w1 shape: (50, 72, 60, 96)
u2 shape: (50, 72, 60, 96)
v2 shape: (50, 84, 60, 96)
w2 shape: (50, 84, 60, 96)
u1 - u2: 0.0


interactive(children=(IntSlider(value=0, description='frame', max=49), IntSlider(value=0, description='x', max…

In [ ]:
# path_invivo = '../../data/PIA/THORAX/P03/h5/P03.h5' 
# path_invivo = '../..//data/paired_invivo/v3_wholeheart_25mm_40ms.h5'
path_result1 ='../../results/Temporal4DFlowNet_20250430-1606/Testset_result_model4_2mm_step2_1606_temporal_mask_2x_test5t12s12s.h5'

# load data
with h5py.File(path_result1, 'r') as h5:
    print(h5.keys())
    u = np.array(h5['u_combined'])
    v = np.array(h5['v_combined'])
    w = np.array(h5['w_combined'])
    # print('vencs:', h5['u_max'][0],  h5['v_max'][0], h5['w_max'][0],)
    if 'mask' not in h5.keys():
        maks = np.ones_like(u)
    else:
        mask = np.array(h5['mask'])
        if mask.ndim == 3:
            mask = mask[np.newaxis, ...]
            mask = np.repeat(mask, u.shape[0], axis=0)


# u *= mask
T, X, Y, Z = u.shape
masking = False

if masking:
    u = u * mask
    v = v * mask
    w = w * mask

@widgets.interact(frame=(0, T-1), x=(0, X-1), axis=[0, 1, 2])
def visualize_lr(frame=0, x=0, axis=0):
    plt.figure(figsize=(12, 7))
    N = 4
    idxs_hr = get_indices(frame, axis, x)

    plt.subplot(1, N, 1)
    plt.imshow(u[idxs_hr])
    plt.title('u')
    plt.axis('off')

    plt.subplot(1, N, 2)
    plt.imshow(v[idxs_hr])
    plt.title('v')
    plt.axis('off')

    plt.subplot(1, N, 3)
    plt.imshow(w[idxs_hr])
    plt.title('v')
    plt.axis('off')

    # plt.subplot(1, N, 4)
    # plt.imshow(mask[idxs_hr])
    # plt.title('mask')
    # # plt.axis('off')

<KeysViewHDF5 ['u_combined', 'v_combined', 'w_combined']>


interactive(children=(IntSlider(value=0, description='frame', max=49), IntSlider(value=0, description='x', max…

In [ ]:
# compare the two predictions

path_result1 ='../../results/Temporal4DFlowNet_20250430-1606/Testset_result_model4_2mm_step2_1606_temporal_mask_2x_test5t12s12s.h5'
path_results2 ='../../results/Temporal4DFlowNet_20250430-1606/Testset_result_model4_2mm_step2_1606_temporal_mask_2x.h5'

# load datapath_result1
with h5py.File(path_result1, 'r') as h5:
    print(h5.keys())
    u = np.array(h5['u_combined'])
    v = np.array(h5['v_combined'])
    w = np.array(h5['w_combined'])

with h5py.File(path_results2, 'r') as h5:
    print(h5.keys())
    u2 = np.array(h5['u_combined'])
    v2 = np.array(h5['v_combined'])
    w2 = np.array(h5['w_combined'])


print(np.sum(np.linalg.norm(v - v2)))

@widgets.interact(frame=(0, T-1), x=(0, X-1), axis=[0, 1, 2])
def visualize_lr(frame=0, x=0, axis=0):
    plt.figure(figsize=(12, 7))
    N = 4
    idxs_hr = get_indices(frame, axis, x)

    plt.subplot(1, N, 1)
    plt.imshow(u[idxs_hr])
    plt.title('u')
    plt.axis('off')

    plt.subplot(1, N, 2)
    plt.imshow(u2[idxs_hr])
    plt.title('u2')
    plt.axis('off')

    plt.subplot(1, N, 3)
    plt.imshow(u[idxs_hr]-u2[idxs_hr])
    plt.title('diff')
    plt.axis('off')

# also check prediction error!


<KeysViewHDF5 ['u_combined', 'v_combined', 'w_combined']>
<KeysViewHDF5 ['u_combined', 'v_combined', 'w_combined']>
0.0


interactive(children=(IntSlider(value=0, description='frame', max=49), IntSlider(value=0, description='x', max…